# Consulta de recomendações a partir do neo4j

## Instalação biblioteca

In [1]:
%pip install py2neo
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importação biblioteca

In [2]:
from py2neo import Graph
from tqdm import tqdm

## Conectando ao neo4j

In [4]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

## Salvando as similaridades de animes

Só rodar a célula de código a seguir caso as similaridades não tenham sido salvas

In [4]:
#resultados = graph.run(
#'''
#MATCH 
#(anime: Anime)
#CALL {
#    WITH anime
#    MATCH
#    (anime) <-[anime1Rating:Rating]- (usersWhoLiked: User) -[usersRatings:Rating]-> (otherAnimes: Anime)
#    RETURN anime AS src_anime, otherAnimes AS target_anime, count(*) as gosteis
#    ORDER BY gosteis DESC
#    LIMIT 3
#}
#WITH src_anime, target_anime, gosteis
#CREATE
#(src_anime) -[:Similar {gosteis: gosteis}]-> (target_anime)
#'''
#)
#resultados

## Execução da consulta

In [5]:
resultados = graph.run(
'''
MATCH (user: User)
CALL {
    WITH user
    MATCH (user) -[user2Ratings:Rating]-> (likedUser2Animes: Anime) -[similarities:Similar]-> (similarAnimes:Anime)
    RETURN similarAnimes.anime_id AS recomended_anime_id, similarAnimes.name AS recomended_anime_name, SUM(similarities.gosteis) AS gosteis
    ORDER BY gosteis DESC
    LIMIT 3
}
RETURN user.user_id AS target_user, recomended_anime_id, recomended_anime_name, gosteis
'''
)
resultados

target_user,recomended_anime_id,recomended_anime_name,gosteis
148,1735,Naruto: Shippuuden,11652
148,1535,Death Note,10146
148,5114,Fullmetal Alchemist: Brotherhood,9902


In [6]:
resultados_list = resultados.data()

print(resultados_list[:5])

[{'target_user': 148, 'recomended_anime_id': 1735, 'recomended_anime_name': 'Naruto: Shippuuden', 'gosteis': 11652}, {'target_user': 148, 'recomended_anime_id': 1535, 'recomended_anime_name': 'Death Note', 'gosteis': 10146}, {'target_user': 148, 'recomended_anime_id': 5114, 'recomended_anime_name': 'Fullmetal Alchemist: Brotherhood', 'gosteis': 9902}, {'target_user': 463, 'recomended_anime_id': 5114, 'recomended_anime_name': 'Fullmetal Alchemist: Brotherhood', 'gosteis': 46936}, {'target_user': 463, 'recomended_anime_id': 2904, 'recomended_anime_name': 'Code Geass: Hangyaku no Lelouch R2', 'gosteis': 28233}]


In [8]:
qnt_recs = len(resultados_list)
print(qnt_recs)

887217


In [11]:
recomendations = {}

for rec in tqdm(resultados_list, total=qnt_recs):
    if rec['target_user'] not in recomendations:
        recomendations[rec['target_user']] = []
        
    recomendations[rec['target_user']].append({
        'recomended_anime_id': rec['recomended_anime_id'],
        'recomended_anime_name': rec['recomended_anime_name']
    })

print(recomendations[148])

100%|██████████| 887217/887217 [00:01<00:00, 462500.16it/s]

295739
